In [5]:
!pip install tensorflow

  Using cached tensorflow-2.17.0-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
  Using cached tensorflow_intel-2.17.0-cp311-cp311-win_amd64.whl.metadata (5.0 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.11.0-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached ml_dtypes-0.4.0-cp311-cp311-win_amd64.whl.metadata (20 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached protobuf-4.25.4-cp310-abi3-win_amd64.whl.metadata (541 bytes)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.16.0-cp311-cp311-win_amd64.whl.metadata (


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install mediapipe

  Using cached mediapipe-0.10.14-cp311-cp311-win_amd64.whl.metadata (9.9 kB)
  Using cached jax-0.4.31-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.4.31-cp311-cp311-win_amd64.whl.metadata (1.0 kB)
  Using cached opencv_contrib_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.4.7-py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached contourpy-1.2.1-cp311-cp311-win_amd64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.53.1-cp311-cp311-win_amd64.whl.metadata (165 kB)
  Using cached kiwisolver-1.4.5-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
Using cached mediapipe-0.10.14-cp311-cp311-win_amd64.whl (50.8 MB)
Using cached sounddevice-0.4.7-py3-none-win_amd64.whl (200 kB)
Using cached jax-0.4.31-py3-none-any.whl (2.0 MB)
Using cached jaxlib-0.4.31-cp311-cp311-win_amd64.whl (56.3 MB)
   -------------------------


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import mediapipe as mp
import os
import time

In [3]:
[dir for dir in os.listdir('data/FinalProcessed') if os.path.isdir(os.path.join('data/FinalProcessed', dir))]

['beautiful', 'blind', 'deaf', 'happy', 'loud', 'quiet']

In [4]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [31]:
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image,results
    

In [32]:
def draw_landmarks(image,results):
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
    #                          mp_drawing.DrawingSpec(color = (80,110,10), thickness = 1, circle_radius = 1),
    #                          mp_drawing.DrawingSpec(color = (80,256,121), thickness = 1, circle_radius = 1))
    # mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
    #                          mp_drawing.DrawingSpec(color = (80,22,10), thickness = 2, circle_radius = 4),
    #                          mp_drawing.DrawingSpec(color = (80,44,121), thickness = 2, circle_radius = 2)
    #                          )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (121,22,76), thickness = 2, circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (121,44,350), thickness = 2, circle_radius = 2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color = (245,117,66), thickness = 2, circle_radius = 4),
                             mp_drawing.DrawingSpec(color = (245,66,230), thickness = 2, circle_radius = 2))

In [67]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
        ret,frame=cap.read()
        image,results = mediapipe_detection(frame,holistic)
        draw_landmarks(image,results)
        cv2.imshow('OpenCV Feed',image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [33]:
def extract_keypoints(results):
    # pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    # face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    # return np.concatenate([pose, face, lh, rh])
    return np.concatenate([lh, rh])

In [34]:
def process_video(video_path, output_folder, holistic):
    cap = cv2.VideoCapture(video_path)
    frame_num = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        _, results = mediapipe_detection(frame, holistic)
        keypoints = extract_keypoints(results)
        frame_folder = os.path.join(output_folder, f'frame_{frame_num:04d}')
        np.save(frame_folder, keypoints)
        frame_num += 1

    cap.release()


In [35]:
def create_output_structure(data_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for adjective in os.listdir(data_folder):
        adjective_folder = os.path.join(data_folder, adjective)
        if os.path.isdir(adjective_folder):
            output_adjective_folder = os.path.join(output_folder, adjective)
            if not os.path.exists(output_adjective_folder):
                os.makedirs(output_adjective_folder)

            for video in os.listdir(adjective_folder):
                video_path = os.path.join(adjective_folder, video)
                video_output_folder = os.path.join(output_adjective_folder, video.split('.')[0])
                if not os.path.exists(video_output_folder):
                    os.makedirs(video_output_folder)

                yield video_path, video_output_folder


In [36]:
def get_video_frame_count(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return frame_count

In [37]:
def calculate_average_frame_count(data_folder):
    total_frames = 0
    num_videos = 0
    
    for root, dirs, files in os.walk(data_folder):
        for file in files:
            if file.endswith(('.MOV')):  # Add other video formats if needed
                video_path = os.path.join(root, file)
                frame_count = get_video_frame_count(video_path)
                total_frames += frame_count
                num_videos += 1
    if num_videos == 0:
        return 0  # Avoid division by zero
    return total_frames // num_videos

In [101]:
calculate_average_frame_count('data/FinalProcessed')

58

In [16]:
def process_videos_to_average_frame_count(data_folder, average_frame_count, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for root, dirs, files in os.walk(data_folder):
        for dir in dirs:
            adjective_folder = os.path.join(output_folder, os.path.relpath(os.path.join(root, dir), data_folder))
            if not os.path.exists(adjective_folder):
                os.makedirs(adjective_folder)
                
        for file in files:
            if file.endswith(('.mp4', '.mov', '.avi', '.mkv', '.MOV')):  # Add other video formats if needed
                video_path = os.path.join(root, file)
                cap = cv2.VideoCapture(video_path)
                frames = []
                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break
                    frames.append(frame)
                
                cap.release()
                
                if len(frames) > average_frame_count:
                    processed_frames = truncate_frames_uniformly(frames, average_frame_count)
                else:
                    processed_frames = pad_frames_uniformly(frames, average_frame_count)
                
                adjective_folder = os.path.join(output_folder, os.path.relpath(root, data_folder))
                video_name = os.path.splitext(file)[0] + '.MOV'
                save_video(processed_frames, os.path.join(adjective_folder, video_name))

In [102]:
def save_video(frames, output_path, fps=30):
    height, width, layers = frames[0].shape
    size = (width, height)
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
    
    for frame in frames:
        out.write(frame)
    out.release()

In [35]:
data_folder = 'data/Augment'
output_folder = 'data/Processed'
average_frame_count = calculate_average_frame_count(data_folder)
print(f'Average frame count: {average_frame_count}')

process_videos_to_average_frame_count(data_folder, average_frame_count, output_folder)

Average frame count: 58


In [106]:
data_folder = 'data/FinalProcessed'
output_folder = 'output_keypoints2'
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for video_path, video_output_folder in create_output_structure(data_folder, output_folder):
        process_video(video_path, video_output_folder, holistic)

print("Keypoints extraction complete.")

Keypoints extraction complete.


In [37]:
!pip install -U scikit-learn

   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   -- ------------------------------------- 0.6/11.0 MB 18.2 MB/s eta 0:00:01
   ---- ----------------------------------- 1.3/11.0 MB 16.4 MB/s eta 0:00:01
   ------ --------------------------------- 1.8/11.0 MB 14.6 MB/s eta 0:00:01
   --------- ------------------------------ 2.5/11.0 MB 14.7 MB/s eta 0:00:01
   ----------- ---------------------------- 3.2/11.0 MB 14.6 MB/s eta 0:00:01
   ------------- -------------------------- 3.8/11.0 MB 14.4 MB/s eta 0:00:01
   --------------- ------------------------ 4.3/11.0 MB 13.7 MB/s eta 0:00:01
   ------------------ --------------------- 5.0/11.0 MB 14.0 MB/s eta 0:00:01
   -------------------- ------------------- 5.6/11.0 MB 13.8 MB/s eta 0:00:01
   ---------------------- ----------------- 6.2/11.0 MB 13.7 MB/s eta 0:00:01
   ------------------------ --------------- 6.8/11.0 MB 14.1 MB/s eta 0:00:01
   -------------------------- ------------- 7.4/11.0 MB 13.4 MB/s eta 0:


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [5]:
actions = [dir for dir in os.listdir('data/FinalProcessed') if os.path.isdir(os.path.join('data/FinalProcessed', dir))]
label_map = {label:num for num,label in enumerate(actions)}
label_map

{'beautiful': 0, 'blind': 1, 'deaf': 2, 'happy': 3, 'loud': 4, 'quiet': 5}

In [6]:
# Define paths and parameters
DATA_PATH = 'output_keypoints2'
sequence_length = 58 
label_map = {label: num for num, label in enumerate(actions)}

sequences, labels = [], []

# Iterate over each action and sequence
for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    
    # List all videos in the current action folder
    videos = [f for f in os.listdir(action_path) if os.path.isdir(os.path.join(action_path, f))]
    
    for video in videos:
        video_path = os.path.join(action_path, video)
        window = []
        
        # Load all keypoint numpy files for this video
        for frame_num in range(sequence_length):
            frame_path = os.path.join(video_path, f'frame_{frame_num:04d}.npy')
            if os.path.exists(frame_path):
                res = np.load(frame_path)
                window.append(res)
            else:
                # Handle case where the frame is missing
                # You can choose to pad with zeros or ignore the video
                window.append(np.zeros((1662,)))  # Assuming keypoints have shape (1662,)
        
        # Check if window is complete
        if len(window) == sequence_length:
            sequences.append(window)
            labels.append(label_map[action])

# Convert lists to numpy arrays
sequences = np.array(sequences)
labels = np.array(labels)

print(f"Sequences shape: {sequences.shape}")
print(f"Labels shape: {labels.shape}")

KeyboardInterrupt: 

In [44]:
X = np.array(sequences)

In [45]:
X.shape

(391, 58, 126)

In [48]:
y = to_categorical(labels).astype(int)

In [49]:
y[0]

array([1, 0, 0, 0, 0, 0])

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.05)

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D, Dropout, Flatten, Dense, BatchNormalization, Input, Reshape
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [7]:
actions = np.array(actions)

In [15]:
height = 126
width = 224
channels = 3
time_steps = 58
num_classes = actions.shape[0]

In [12]:
num_classes

6

In [14]:
input_dir = 'data/FinalProcessed'
output_dir = 'data/FinalCompressed'
new_width = 224  # Desired width
new_height = 126 # Desired height

# Create output directory if it does not exist
os.makedirs(output_dir, exist_ok=True)

# Function to resize video frames
def resize_video(video_path, output_path, width, height):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # Codec for .mp4
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_size = (width, height)
    out = cv2.VideoWriter(output_path, fourcc, fps, frame_size)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        resized_frame = cv2.resize(frame, frame_size, interpolation=cv2.INTER_AREA)
        out.write(resized_frame)
    
    cap.release()
    out.release()

# Iterate through each adjective folder
for adjective_folder in os.listdir(input_dir):
    input_adjective_folder = os.path.join(input_dir, adjective_folder)
    output_adjective_folder = os.path.join(output_dir, adjective_folder)
    os.makedirs(output_adjective_folder, exist_ok=True)

    # Process each video in the adjective folder
    for video_file in os.listdir(input_adjective_folder):
        video_path = os.path.join(input_adjective_folder, video_file)
        output_path = os.path.join(output_adjective_folder, video_file)
        resize_video(video_path, output_path, new_width, new_height)
        print(f'Resized and saved {video_file} to {output_path}')


Resized and saved MVI_9569.MOV to data/FinalCompressed\beautiful\MVI_9569.MOV
Resized and saved MVI_9569_augment_0000.MOV to data/FinalCompressed\beautiful\MVI_9569_augment_0000.MOV
Resized and saved MVI_9569_augment_0001.MOV to data/FinalCompressed\beautiful\MVI_9569_augment_0001.MOV
Resized and saved MVI_9569_augment_0001_augment_0000.MOV to data/FinalCompressed\beautiful\MVI_9569_augment_0001_augment_0000.MOV
Resized and saved MVI_9569_augment_0003.MOV to data/FinalCompressed\beautiful\MVI_9569_augment_0003.MOV
Resized and saved MVI_9569_augment_0003_augment_0000.MOV to data/FinalCompressed\beautiful\MVI_9569_augment_0003_augment_0000.MOV
Resized and saved MVI_9569_augment_0004.MOV to data/FinalCompressed\beautiful\MVI_9569_augment_0004.MOV
Resized and saved MVI_9569_augment_0004_augment_0000.MOV to data/FinalCompressed\beautiful\MVI_9569_augment_0004_augment_0000.MOV
Resized and saved MVI_9570.MOV to data/FinalCompressed\beautiful\MVI_9570.MOV
Resized and saved MVI_9570_augment_000

In [21]:
def extract_frames(video_path):
    cap = cv2.VideoCapture(video_path)  # Open the video file
    frames = []  # List to store frames
    while cap.isOpened():
        ret, frame = cap.read()  # Read a frame from the video
        if not ret:
            break  # Exit if no frame is read (end of video)
        frames.append(frame)  # Add frame to the list
    cap.release()  # Release the video capture object
    
    return np.array(frames)  # Return the list of frames as a NumPy array

In [22]:
def load_data(data_folder):
    X, y = [] , []  # Lists to store feature sequences and labels
    class_labels = sorted(os.listdir(data_folder))  # Get all class labels (subfolder names)
    label_to_index = {label: idx for idx, label in enumerate(class_labels)}  # Create a label-to-index map
    
    for label_folder in class_labels:
        folder_path = os.path.join(data_folder, label_folder)  # Path to the label folder
        for video_file in os.listdir(folder_path):
            video_path = os.path.join(folder_path, video_file)  # Path to the video file
            frames = extract_frames(video_path)  # Extract and resize frames
            X.append(frames)  # Add the frame sequence to X
            y.append(label_to_index[label_folder])  # Add the corresponding label index to y
    
    return np.array(X), np.array(y)

In [23]:
X, y = load_data('data/FinalCompressed')

MemoryError: Unable to allocate 4.68 MiB for an array with shape (58, 126, 224, 3) and data type uint8

In [52]:
class DataGenerator(Sequence):
    def __init__(self, data_folder, labels, batch_size=16, sequence_length=58, shuffle=True, num_classes=6):
        self.data_folder = data_folder
        self.labels = labels
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.shuffle = shuffle
        self.num_classes = num_classes
        self.indexes = np.arange(len(self.labels))
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.floor(len(self.labels) / self.batch_size))
    
    def __getitem__(self, index):
        batch_indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        X, y = self.__data_generation(batch_indexes)
        return X, y
    
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)
    
    def __data_generation(self, batch_indexes):
        X_batch = np.empty((self.batch_size, self.sequence_length, 126, 224, 3), dtype=np.uint8)
        y_batch = np.empty((self.batch_size), dtype=int)
        
        for i, idx in enumerate(batch_indexes):
            video_file = self.labels[idx][0]
            label = self.labels[idx][1]
            
            video_path = os.path.join(self.data_folder, video_file)
            frames = extract_frames(video_path)
            
            X_batch[i,] = frames
            y_batch[i] = label
        
        # One-hot encode the labels
        y_batch = to_categorical(y_batch, num_classes=self.num_classes)
        
        return X_batch, y_batch


In [53]:
data_folder = 'data/FinalCompressed'
class_labels = sorted(os.listdir(data_folder))
label_to_index = {label: idx for idx, label in enumerate(class_labels)}

# Prepare the list of video paths and labels
labels = []
for label_folder in class_labels:
    folder_path = os.path.join(data_folder, label_folder)
    for video_file in os.listdir(folder_path):
        labels.append((os.path.join(label_folder, video_file), label_to_index[label_folder]))


In [54]:
from sklearn.model_selection import train_test_split

labels_train, labels_test = train_test_split(labels, test_size=0.05, random_state=42)

In [55]:
from tensorflow.keras.utils import to_categorical

# Assuming labels are integers representing classes
y_train = to_categorical([label[1] for label in labels_train], num_classes=6)
y_test = to_categorical([label[1] for label in labels_test], num_classes=6)


In [60]:
train_gen = DataGenerator(data_folder='data/FinalCompressed', labels=labels_train, batch_size=16)
val_gen = DataGenerator(data_folder='data/FinalCompressed', labels=labels_test, batch_size=16)

In [61]:
batch_size = 16
time_steps = 58  # Number of frames per video
width, height = 224, 126  # Frame dimensions
channels = 3  # Number of channels (RGB)
num_classes = 6  # Number of adjectives/classes

In [62]:
convlstm = Sequential()

convlstm.add(ConvLSTM2D(filters=32, kernel_size=(3, 3),
                     input_shape=(time_steps, height, width, channels),
                     padding='same', return_sequences=True, activation='relu'))
convlstm.add(BatchNormalization())

convlstm.add(ConvLSTM2D(filters=64, kernel_size=(3, 3),
                     padding='same', return_sequences=False, activation='relu'))
convlstm.add(BatchNormalization())

convlstm.add(Flatten())
convlstm.add(Dense(128, activation='relu'))
convlstm.add(Dense(num_classes, activation='softmax'))


In [63]:
convlstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [64]:
history = convlstm.fit(
    train_gen,
    validation_data=val_gen,
    epochs=20
)

TypeError: extract_frames() takes 1 positional argument but 3 were given

In [40]:
model = Sequential()
model.add(Input(shape=(58, 126)))
model.add(LSTM(64, return_sequences=True, activation='relu'))
model.add(Dropout(0.3))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(Dropout(0.3))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [13]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 58, 64)              │          48,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 58, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 58, 128)             │          98,816 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 58, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 214,086 (836.27 KB)

 Trainable params: 214,086 (836.27 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.1, random_state=42)

In [41]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# # Early Stopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


In [157]:
history = model.fit(X_train, y_train, epochs=400)

Epoch 1/400
12/12 ━━━━━━━━━━━━━━━━━━━━ 5s 94ms/step - categorical_accuracy: 0.3617 - loss: 1.5389
Epoch 2/400
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - categorical_accuracy: 0.3235 - loss: 1.3210
Epoch 3/400
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 91ms/step - categorical_accuracy: 0.3819 - loss: 1.2332
Epoch 4/400
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - categorical_accuracy: 0.4176 - loss: 1.2124
Epoch 5/400
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 95ms/step - categorical_accuracy: 0.4050 - loss: 1.2250
Epoch 6/400
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 89ms/step - categorical_accuracy: 0.3898 - loss: 1.2639
Epoch 7/400
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 87ms/step - categorical_accuracy: 0.4262 - loss: 1.1618
Epoch 8/400
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - categorical_accuracy: 0.3459 - loss: 1.2094
Epoch 9/400
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 90ms/step - categorical_accuracy: 0.4423 - loss: 1.2098
Epoch 10/400
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step - categorical_accuracy: 0.4425 - loss: 1.1669
Epoch 11/400
12/12 

In [57]:
res = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


In [60]:
actions[np.argmax(res[4])]

'deaf'

In [61]:
actions[np.argmax(y_test[4])]

'beautiful'

In [163]:
model.save('lstm.h5')

In [62]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [67]:
yhat = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


In [68]:
ytrue = np.argmax(y_test,axis = 1).tolist()
yhat = np.argmax(yhat,axis = 1).tolist()

In [69]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[17,  0],
        [ 0,  3]],

       [[18,  0],
        [ 0,  2]],

       [[16,  0],
        [ 0,  4]],

       [[15,  0],
        [ 0,  5]],

       [[17,  0],
        [ 0,  3]],

       [[17,  0],
        [ 0,  3]]], dtype=int64)

In [66]:
model.load_weights('lstm.h5')

In [70]:
from scipy import stats

In [71]:
colors = [(245,117,16), (117,245,16), (16,117,245), (245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
sequence = []
sentence = []
predictions = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-58:]
        
        if len(sequence) == 58:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti